In [1]:
# CSL Paper: Dimensional speech emotion recognition from acoustic and text
# Changelog:
# 2019-09-01: initial version
# 2019-10-06: optimizer MTL parameters with linear search (in progress)
# 2019-12-21: update file for CSL journal
#             this python use following feature: mean, std, silence

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from attention_helper import AttentionDecoder
#from read_csv import load_features

import random as rn
import tensorflow as tf

# import gensim
# from gensim.models import Word2Vec
# from gensim.models.keyedvectors import KeyedVectors
import codecs

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# x_text = np.load('../improv_loso/data/x_text.npy')

In [4]:
# x_text.shape

In [5]:
# load data - label
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_train = pd.read_csv(list_path_train, index_col=None)

# vad_train = [list_train['v'], list_train['a'], list_train['d']]
# vad_train = np.array(vad_train).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

## Text processing train
list_file = pd.concat([list_train, list_test])
text = list_file['transcript']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
nb_words = len(word_index) + 1
print('Found %s unique tokens' % len(word_index))

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_text = []
x_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# x_text = np.load('improv_loso/data/x_text.npy')
# x_train_text = x_text[:5744] 
# x_test_text = x_text[5744:]

# vad = np.load('improv_loso/data/vad.npy')
# vad_train = vad[:5744]
# vad_test = vad[5744:]

EMBEDDING_DIM = 300
# nb_words = 3438
# MAX_SEQUENCE_LENGTH = 300

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
file_loc =  '../../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

Found 3427 unique tokens
../../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 167


In [6]:
vad = np.load('../improv_loso/data/vad.npy')

In [7]:
# standardization
scaled_feature = False

# set Dropout
do = 0.3

# standardized label
scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad


# Concordance correlation coefficient (CCC)-based loss function
# using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [8]:
# split train/test
split = 5733
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)

In [9]:
# API model, if use RNN, first two rnn layer must return_sequences=True
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
#                     weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model1 = text_model1()

In [10]:
# main function for for LOSO
def main(time):
    model_1 = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [x_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [x_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict(x_text[4586:], batch_size=32)
    np.save('5733npy-WE/text_WE5733-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 2580).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('5733npy-WE/text_WE-5733.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 88s 19ms/step - loss: 2.5964 - v_loss: 0.7531 - a_loss: 0.9095 - d_loss: 0.9252 - v_ccc: 0.2445 - a_ccc: 0.0870 - d_ccc: 0.0721 - val_loss: 2.6695 - val_v_loss: 0.8463 - val_a_loss: 0.9085 - val_d_loss: 0.9119 - val_v_ccc: 0.1527 - val_a_ccc: 0.0908 - val_d_ccc: 0.0869
Epoch 2/50
4586/4586 [==============================] - 89s 20ms/step - loss: 1.9737 - v_loss: 0.5517 - a_loss: 0.6924 - d_loss: 0.7329 - v_ccc: 0.4483 - a_ccc: 0.3089 - d_ccc: 0.2691 - val_loss: 2.6847 - val_v_loss: 0.8657 - val_a_loss: 0.9022 - val_d_loss: 0.9139 - val_v_ccc: 0.1328 - val_a_ccc: 0.0973 - val_d_ccc: 0.0852
Epoch 3/50
4586/4586 [==============================] - 89s 19ms/step - loss: 1.7271 - v_loss: 0.4742 - a_loss: 0.6071 - d_loss: 0.6454 - v_ccc: 0.5251 - a_ccc: 0.3925 - d_ccc: 0.3554 - val_loss: 2.6500 - val_v_loss: 0

Epoch 11/50
4586/4586 [==============================] - 91s 20ms/step - loss: 0.9504 - v_loss: 0.2577 - a_loss: 0.3414 - d_loss: 0.3497 - v_ccc: 0.7419 - a_ccc: 0.6576 - d_ccc: 0.6501 - val_loss: 2.7527 - val_v_loss: 0.8515 - val_a_loss: 0.9339 - val_d_loss: 0.9644 - val_v_ccc: 0.1466 - val_a_ccc: 0.0660 - val_d_ccc: 0.0346
Epoch 12/50
1433/1433 [==============================] - 8s 5ms/step
[0.21076522767543793, 0.10326659679412842, 0.0849044993519783]
1
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 93s 20ms/step - loss: 2.5413 - v_loss: 0.7431 - a_loss: 0.8872 - d_loss: 0.9142 - v_ccc: 0.2591 - a_ccc: 0.1132 - d_ccc: 0.0865 - val_loss: 2.6023 - val_v_loss: 0.8359 - val_a_loss: 0.8650 - val_d_loss: 0.8988 - val_v_ccc: 0.1634 - val_a_ccc: 0.1344 - val_d_ccc: 0.0999
Epoch 2/50
4586/4586 [==============================] - 90s 20ms/step - loss: 1.9662 - v_loss: 0.5524 - a_loss: 0.6844 - d_loss: 0.7312 - v_ccc: 0.4486 - a_ccc: 0.31

4586/4586 [==============================] - 89s 19ms/step - loss: 0.9104 - v_loss: 0.2564 - a_loss: 0.3214 - d_loss: 0.3322 - v_ccc: 0.7438 - a_ccc: 0.6783 - d_ccc: 0.6675 - val_loss: 2.7895 - val_v_loss: 0.8903 - val_a_loss: 0.9311 - val_d_loss: 0.9674 - val_v_ccc: 0.1086 - val_a_ccc: 0.0693 - val_d_ccc: 0.0325
Epoch 13/50
1433/1433 [==============================] - 8s 5ms/step
[0.18434883654117584, 0.16867013275623322, 0.10864317417144775]
3
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 92s 20ms/step - loss: 2.5271 - v_loss: 0.7394 - a_loss: 0.8856 - d_loss: 0.9027 - v_ccc: 0.2603 - a_ccc: 0.1152 - d_ccc: 0.0974 - val_loss: 2.6448 - val_v_loss: 0.8643 - val_a_loss: 0.8970 - val_d_loss: 0.8804 - val_v_ccc: 0.1345 - val_a_ccc: 0.1024 - val_d_ccc: 0.1183
Epoch 2/50
4586/4586 [==============================] - 89s 19ms/step - loss: 1.9672 - v_loss: 0.5519 - a_loss: 0.6844 - d_loss: 0.7303 - v_ccc: 0.4472 - a_ccc: 0.3155 - d_ccc:

1433/1433 [==============================] - 8s 5ms/step
[0.18535013496875763, 0.16675272583961487, 0.13545702397823334]
5
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 98s 21ms/step - loss: 2.5333 - v_loss: 0.7394 - a_loss: 0.8896 - d_loss: 0.9057 - v_ccc: 0.2590 - a_ccc: 0.1115 - d_ccc: 0.0962 - val_loss: 2.7020 - val_v_loss: 0.8663 - val_a_loss: 0.9022 - val_d_loss: 0.9311 - val_v_ccc: 0.1330 - val_a_ccc: 0.0968 - val_d_ccc: 0.0683
Epoch 2/50
4586/4586 [==============================] - 94s 20ms/step - loss: 1.9790 - v_loss: 0.5554 - a_loss: 0.6916 - d_loss: 0.7311 - v_ccc: 0.4445 - a_ccc: 0.3077 - d_ccc: 0.2688 - val_loss: 2.7131 - val_v_loss: 0.8383 - val_a_loss: 0.9378 - val_d_loss: 0.9352 - val_v_ccc: 0.1610 - val_a_ccc: 0.0619 - val_d_ccc: 0.0640
Epoch 3/50
4586/4586 [==============================] - 93s 20ms/step - loss: 1.7273 - v_loss: 0.4843 - a_loss: 0.5975 - d_loss: 0.6470 - v_ccc: 0.5165 - a_ccc: 0.4024 - d_ccc: 

4586/4586 [==============================] - 90s 20ms/step - loss: 0.9464 - v_loss: 0.2597 - a_loss: 0.3368 - d_loss: 0.3486 - v_ccc: 0.7398 - a_ccc: 0.6632 - d_ccc: 0.6506 - val_loss: 2.7617 - val_v_loss: 0.8851 - val_a_loss: 0.9112 - val_d_loss: 0.9638 - val_v_ccc: 0.1135 - val_a_ccc: 0.0890 - val_d_ccc: 0.0358
Epoch 12/50
4586/4586 [==============================] - 86s 19ms/step - loss: 0.8941 - v_loss: 0.2454 - a_loss: 0.3206 - d_loss: 0.3283 - v_ccc: 0.7543 - a_ccc: 0.6797 - d_ccc: 0.6720 - val_loss: 2.8222 - val_v_loss: 0.9316 - val_a_loss: 0.9187 - val_d_loss: 0.9706 - val_v_ccc: 0.0672 - val_a_ccc: 0.0814 - val_d_ccc: 0.0292
Epoch 13/50
1433/1433 [==============================] - 8s 6ms/step
[0.1816643923521042, 0.21229402720928192, 0.12626121938228607]
7
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 94s 21ms/step - loss: 2.5804 - v_loss: 0.7564 - a_loss: 0.9054 - d_loss: 0.9196 - v_ccc: 0.2449 - a_ccc: 0.0942 - d_ccc:

...

4586/4586 [==============================] - 90s 20ms/step - loss: 1.1345 - v_loss: 0.3082 - a_loss: 0.4040 - d_loss: 0.4208 - v_ccc: 0.6912 - a_ccc: 0.5961 - d_ccc: 0.5782 - val_loss: 2.7838 - val_v_loss: 0.8997 - val_a_loss: 0.9210 - val_d_loss: 0.9608 - val_v_ccc: 0.0987 - val_a_ccc: 0.0789 - val_d_ccc: 0.0386
Epoch 9/50
4586/4586 [==============================] - 86s 19ms/step - loss: 1.0630 - v_loss: 0.2899 - a_loss: 0.3773 - d_loss: 0.3955 - v_ccc: 0.7114 - a_ccc: 0.6216 - d_ccc: 0.6039 - val_loss: 2.7719 - val_v_loss: 0.8417 - val_a_loss: 0.9308 - val_d_loss: 0.9975 - val_v_ccc: 0.1568 - val_a_ccc: 0.0690 - val_d_ccc: 0.0023
Epoch 10/50
4586/4586 [==============================] - 90s 20ms/step - loss: 1.0040 - v_loss: 0.2707 - a_loss: 0.3586 - d_loss: 0.3760 - v_ccc: 0.7288 - a_ccc: 0.6420 - d_ccc: 0.6252 - val_loss: 2.7859 - val_v_loss: 0.8767 - val_a_loss: 0.9336 - val_d_loss: 0.9739 - val_v_ccc: 0.1217 - val_a_ccc: 0.0668 - val_d_ccc: 0.0256
Epoch 11/50
4586/4586 [=========

4586/4586 [==============================] - 92s 20ms/step - loss: 1.2129 - v_loss: 0.3376 - a_loss: 0.4294 - d_loss: 0.4527 - v_ccc: 0.6628 - a_ccc: 0.5727 - d_ccc: 0.5516 - val_loss: 2.7887 - val_v_loss: 0.8722 - val_a_loss: 0.9349 - val_d_loss: 0.9795 - val_v_ccc: 0.1263 - val_a_ccc: 0.0649 - val_d_ccc: 0.0201
Epoch 8/50
4586/4586 [==============================] - 91s 20ms/step - loss: 1.1348 - v_loss: 0.3139 - a_loss: 0.4031 - d_loss: 0.4155 - v_ccc: 0.6853 - a_ccc: 0.5962 - d_ccc: 0.5836 - val_loss: 2.7601 - val_v_loss: 0.8751 - val_a_loss: 0.9200 - val_d_loss: 0.9630 - val_v_ccc: 0.1236 - val_a_ccc: 0.0797 - val_d_ccc: 0.0365
Epoch 9/50
4586/4586 [==============================] - 86s 19ms/step - loss: 1.0572 - v_loss: 0.2984 - a_loss: 0.3703 - d_loss: 0.3895 - v_ccc: 0.7023 - a_ccc: 0.6295 - d_ccc: 0.6110 - val_loss: 2.7361 - val_v_loss: 0.8609 - val_a_loss: 0.8956 - val_d_loss: 0.9769 - val_v_ccc: 0.1375 - val_a_ccc: 0.1042 - val_d_ccc: 0.0221
Epoch 10/50
4586/4586 [==========

4586/4586 [==============================] - 90s 20ms/step - loss: 1.2017 - v_loss: 0.3313 - a_loss: 0.4270 - d_loss: 0.4481 - v_ccc: 0.6695 - a_ccc: 0.5752 - d_ccc: 0.5536 - val_loss: 2.7348 - val_v_loss: 0.8714 - val_a_loss: 0.9278 - val_d_loss: 0.9328 - val_v_ccc: 0.1267 - val_a_ccc: 0.0718 - val_d_ccc: 0.0667
Epoch 8/50
4586/4586 [==============================] - 91s 20ms/step - loss: 1.1404 - v_loss: 0.3091 - a_loss: 0.4060 - d_loss: 0.4241 - v_ccc: 0.6907 - a_ccc: 0.5943 - d_ccc: 0.5746 - val_loss: 2.7663 - val_v_loss: 0.8679 - val_a_loss: 0.9136 - val_d_loss: 0.9825 - val_v_ccc: 0.1304 - val_a_ccc: 0.0861 - val_d_ccc: 0.0172
Epoch 9/50
4586/4586 [==============================] - 91s 20ms/step - loss: 1.0656 - v_loss: 0.2862 - a_loss: 0.3828 - d_loss: 0.3964 - v_ccc: 0.7131 - a_ccc: 0.6182 - d_ccc: 0.6031 - val_loss: 2.7783 - val_v_loss: 0.8613 - val_a_loss: 0.9138 - val_d_loss: 1.0024 - val_v_ccc: 0.1375 - val_a_ccc: 0.0862 - val_d_ccc: -0.0020
Epoch 10/50
4586/4586 [=========

4586/4586 [==============================] - 85s 19ms/step - loss: 1.1268 - v_loss: 0.3041 - a_loss: 0.4055 - d_loss: 0.4175 - v_ccc: 0.6952 - a_ccc: 0.5953 - d_ccc: 0.5827 - val_loss: 2.7132 - val_v_loss: 0.8548 - val_a_loss: 0.9011 - val_d_loss: 0.9548 - val_v_ccc: 0.1438 - val_a_ccc: 0.0987 - val_d_ccc: 0.0442
Epoch 9/50
4586/4586 [==============================] - 90s 20ms/step - loss: 1.0649 - v_loss: 0.2899 - a_loss: 0.3803 - d_loss: 0.3951 - v_ccc: 0.7093 - a_ccc: 0.6198 - d_ccc: 0.6060 - val_loss: 2.7865 - val_v_loss: 0.8847 - val_a_loss: 0.9107 - val_d_loss: 0.9894 - val_v_ccc: 0.1138 - val_a_ccc: 0.0895 - val_d_ccc: 0.0102
Epoch 10/50
4586/4586 [==============================] - 90s 20ms/step - loss: 1.0022 - v_loss: 0.2753 - a_loss: 0.3568 - d_loss: 0.3722 - v_ccc: 0.7259 - a_ccc: 0.6434 - d_ccc: 0.6286 - val_loss: 2.8130 - val_v_loss: 0.8665 - val_a_loss: 0.9392 - val_d_loss: 1.0055 - val_v_ccc: 0.1320 - val_a_ccc: 0.0607 - val_d_ccc: -0.0056
Epoch 11/50
1433/1433 [========

4586/4586 [==============================] - 85s 19ms/step - loss: 1.0173 - v_loss: 0.2714 - a_loss: 0.3692 - d_loss: 0.3761 - v_ccc: 0.7289 - a_ccc: 0.6307 - d_ccc: 0.6230 - val_loss: 2.7874 - val_v_loss: 0.8731 - val_a_loss: 0.9333 - val_d_loss: 0.9787 - val_v_ccc: 0.1254 - val_a_ccc: 0.0667 - val_d_ccc: 0.0206
Epoch 11/50
4586/4586 [==============================] - 85s 19ms/step - loss: 0.9595 - v_loss: 0.2603 - a_loss: 0.3433 - d_loss: 0.3556 - v_ccc: 0.7395 - a_ccc: 0.6565 - d_ccc: 0.6445 - val_loss: 2.8092 - val_v_loss: 0.8911 - val_a_loss: 0.9413 - val_d_loss: 0.9745 - val_v_ccc: 0.1074 - val_a_ccc: 0.0585 - val_d_ccc: 0.0249
Epoch 12/50
4586/4586 [==============================] - 85s 19ms/step - loss: 0.9163 - v_loss: 0.2492 - a_loss: 0.3241 - d_loss: 0.3443 - v_ccc: 0.7513 - a_ccc: 0.6764 - d_ccc: 0.6560 - val_loss: 2.8623 - val_v_loss: 0.9075 - val_a_loss: 0.9507 - val_d_loss: 1.0024 - val_v_ccc: 0.0912 - val_a_ccc: 0.0493 - val_d_ccc: -0.0028
Epoch 13/50
4586/4586 [=======

In [11]:
# hist1 = model1.fit(x_text[:split], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(x_text[split:], vad[split:].T.tolist())
# print(eval_metrik1[-3:])

# # make prediction
# predict = model1.predict(x_text[4596:], batch_size=32)
# np.save('text_glove_npy_loso', 
#          np.array(predict).reshape(3, 2570).T)